<a href="https://colab.research.google.com/github/Sette/fifa-23-sorteio/blob/telegram_send/Fifa_23_sorteio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot pdfkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.0/549.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
!sudo apt update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [734 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [731 kB]
Fetched 1,802 kB in 1s (1,420 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
8 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [ ]:
!apt install wkhtmltopdf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon bind9-host bind9-libs geoclue-2.0 glib-networking
  glib-networking-common glib-networking-services gsettings-desktop-schemas
  iio-sensor-proxy libavahi-core7 libavahi-glib1 libdaemon0 libevdev2
  libfontenc1 libgudev-1.0-0 libhyphen0 libinput-bin libinput10
  libjson-glib-1.0-0 libjson-glib-1.0-common liblmdb0 libmaxminddb0
  libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200
  libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5 libqt5printsupport5
  libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1
  libsoup2.4-common libwacom-bin libwacom-common libwacom9 libwoff1
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-randr0
  libxcb-render

In [ ]:
import os
from urllib.request import urlopen,Request
import pandas as pd
from bs4 import BeautifulSoup
import json
import pdfkit
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
import random
import numpy as np
from IPython.core.display import display,HTML

from google.colab.patches import cv2_imshow
import cv2
# checking if the directory demo_folder2
# exist or not.
from tqdm.notebook import tqdm
import requests

import time
from telegram.ext import *


In [ ]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

def get_teams(teams_shape=50):

  user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
  headers={'User-Agent':user_agent,}

  START_URL = "https://www.fifaindex.com/pt-br/teams"

  request=Request(START_URL,None,headers) #The assembled request
  response = urlopen(request)
  data = response.read() # The data u need

  def quant_pags(START_URL):
    """
        FUNÇÃO QUE PEGA VIA CRAWLER A QUANTIDADE DE PÁGINAS A SEREM EXPLORADAS PELO ALGORITMO
    """
    request=Request(START_URL,None,headers) #The assembled request
    response = urlopen(request)
    html = response.read() # The data u need
    bsObj = BeautifulSoup(html, "html5lib")

    links = bsObj.findAll("a", {"class": "page-link"})

    for link in links:
      if (link.getText()) == "Last":
        print(link.attrs['href'])
        cont = int(link.attrs['href'].split("=")[-1:][0])

    return cont

  def get_teams_from_page_link(link):
    request=Request(link,None,headers) #The assembled request
    response = urlopen(request)
    html = response.read() # The data u need
    bsObj = BeautifulSoup(html, "html5lib")




    names = bsObj.findAll("td", {"data-title": "Nome"})
    leagues = bsObj.findAll("a", {"class": "link-league"})
    att	= bsObj.findAll("td", {"data-title": "ATA"})
    mid	= bsObj.findAll("td", {"data-title": "MEI"})
    defense = bsObj.findAll("td", {"data-title": "DEF"})
    over = bsObj.findAll("td", {"data-title": "GER"})
    img = bsObj.findAll("img", {"class": "team size-5"})


    df = pd.DataFrame({"Name":names,"League":leagues,"Att":att,"Mid":mid,"Def":defense,"Over":over,'Img':img})




    df["Name"] = df["Name"].apply(lambda x: x.getText())
    df["League"] = df["League"].apply(lambda x: x.getText())
    df["Att"] = df["Att"].apply(lambda x: x.getText())
    df["Mid"] = df["Mid"].apply(lambda x: x.getText())
    df["Def"] = df["Def"].apply(lambda x: x.getText())
    df["Over"] = df["Over"].apply(lambda x: x.getText())
    df["Img"] = df["Img"].apply(lambda x: x['src'])

    return df



  pages = quant_pags(START_URL)

  def download_teams(pages):
    return pd.concat([get_teams_from_page_link(f'{START_URL}/?page={page}') for page in  range(1,pages+1)])


  teams = download_teams(pages)

  teams.Att = teams.Att.astype(int)
  teams.Mid = teams.Mid.astype(int)
  teams.Def = teams.Def.astype(int)

  teams = teams.reset_index()

  teams.drop(columns="index",inplace=True)

  womens_leagues = ["Women's","Féminine"]

  def get_sex_league(team):
    if any(x in team.League for x in womens_leagues):
      return "F"

    return "M"

  def download_image(image_link):

    if not os.path.isdir("imagens"):

      # if the demo_folder2 directory is
      # not present then create it.
      os.makedirs("imagens")

    img_name = image_link.split('/')[-1]
    # img_path = os.path.join('imagens',img_name)
    # img_data = requests.get(image_link).content
    # with open(img_path, 'wb') as handler:
    #   handler.write(img_data)



    return img_name


  teams["Sex"] = [get_sex_league(team) for i,team in teams.iterrows()]

  # teams.drop()


  teams = teams[teams['League'] != 'Libertadores']


  teams=teams.head(teams_shape)

  tqdm.pandas()
  # teams['Img'] = teams.Img.progress_apply(lambda x: download_image(x))



  teams_list = [[team[2],team[3],team[4]] for team in teams.values]


  distance_array = pairwise_distances(teams_list, metric='euclidean')

  return teams,distance_array




In [ ]:
class Telegram_bot():
  def __init__(self):
    self.token = '6184052476:AAGE-wl9tiwRPlKwjXn51fr1Crx_1Pq0qZg'
    # self.chat_id = self.get_chat_id(self.token)
    self.chat_id = '-1001833850465'


  # mostra o id do último grupo adicionado
  def last_chat_id(self):
      try:
          url = "https://api.telegram.org/bot{}/getUpdates".format(self.token)
          response = requests.get(url)
          if response.status_code == 200:
              json_msg = response.json()
              for json_result in reversed(json_msg['result']):
                print(json_result)


              print('Nenhum grupo encontrado')
          else:
              print('A resposta falhou, código de status: {}'.format(response.status_code))
      except Exception as e:
          print("Erro no getUpdates:", e)

  # enviar mensagens utilizando o bot para um chat específico
  def send_message(self,msg):
      try:
          data = {"chat_id": self.chat_id, "text": msg}
          url = "https://api.telegram.org/bot{}/sendMessage".format(self.token)
          requests.post(url, data)
      except Exception as e:
          print("Erro no sendMessage:", e)



  def send_file(self):

    base_url = "https://api.telegram.org/bot6184052476:AAGE-wl9tiwRPlKwjXn51fr1Crx_1Pq0qZg/sendDocument"

    my_file = open("/content/out.pdf", "rb")

    # parameters = {
    #     "chat_id" : "-897907393",
    #     "caption" : "Sorteio do fifa 23"
    # }

    parameters = {
        "chat_id" : self.chat_id,
        "caption" : "Sorteio do fifa 23"
    }




    files = {
        "document" : my_file
    }

    resp = requests.get(base_url, data = parameters, files=files)
    print(resp.text)




  def get_chat_id(self,token):
    # id do chat que será enviado as mensagens
    chat_id = self.last_chat_id()
    return chat_id


In [ ]:
bot = Telegram_bot()

In [ ]:
bot.last_chat_id()

{'update_id': 923803690, 'message': {'message_id': 49, 'from': {'id': 5788084723, 'is_bot': False, 'first_name': 'Bruno', 'last_name': 'Sette', 'username': 'settebr', 'language_code': 'pt-br'}, 'chat': {'id': -982265313, 'title': 'Sorteio Fifa 23', 'type': 'group', 'all_members_are_administrators': True}, 'date': 1690151236, 'new_chat_participant': {'id': 6184052476, 'is_bot': True, 'first_name': 'FIFA', 'username': 'fifa_sette_sorteio_bot'}, 'new_chat_member': {'id': 6184052476, 'is_bot': True, 'first_name': 'FIFA', 'username': 'fifa_sette_sorteio_bot'}, 'new_chat_members': [{'id': 6184052476, 'is_bot': True, 'first_name': 'FIFA', 'username': 'fifa_sette_sorteio_bot'}]}}
{'update_id': 923803689, 'my_chat_member': {'chat': {'id': -982265313, 'title': 'Sorteio Fifa 23', 'type': 'group', 'all_members_are_administrators': True}, 'from': {'id': 5788084723, 'is_bot': False, 'first_name': 'Bruno', 'last_name': 'Sette', 'username': 'settebr', 'language_code': 'pt-br'}, 'date': 1690151236, 'ol

In [ ]:
def sorteia(teams,distance_array,threshold = 3.5):


  sorteios = 10

  def get_team(df,index):
    return df.iloc[index]


  def sorteia_peso(teams,distance_array,threshold):
    while True:


      rand_idx = random.randrange(distance_array.shape[0])
      rand_idy = random.randrange(distance_array.shape[1])

      random_num = distance_array[rand_idx][rand_idy]
      sorteio = random_num

      #
      if sorteio <= threshold:
        time1,time2 = get_team(teams,rand_idx),get_team(teams,rand_idy)
        return time1,time2


  def sorteia_confronto(teams):
    while True:
      time1,time2 = sorteia_peso(teams,distance_array,threshold)
      if time1.Sex == time2.Sex and time1.Name != time2.Name:
        return [time1,time2]


  def show_team(team,telegram_bot):
    # img = cv2.imread(team['Img'], cv2.IMREAD_UNCHANGED)
    # cv2_imshow(img)

    # print(f"Nome: {team['Name']}")
    # print(f"Liga: {team['League']}")
    # print(f"Att {team['Att']} - Mid {team['Mid']} - Def {team['Def']}")

    telegram_bot.send_message(f"Nome: {team['Name']}")
    telegram_bot.send_message(f"Liga: {team['League']}")
    telegram_bot.send_message(f"Att {team['Att']} - Mid {team['Mid']} - Def {team['Def']}")



  telegram_bot = Telegram_bot()

  print(telegram_bot.chat_id)
  confrontos = []

  for i in range(sorteios):
    confronto = sorteia_confronto(teams)

    confrontos.append(np.concatenate(confronto))


    # show_team(confronto[0],telegram_bot)
    # show_team(confronto[1],telegram_bot)
    # time.sleep(2)

  confrontos = pd.DataFrame(confrontos,columns=["Nome 1", "Liga 1", "Ataque 1", "Meio 1", "Defesa 1", "Geral 1" ,"Logo 1", "Sexo 1","Nome 2", "Liga 2", "Ataque 2", "Meio 2", "Defesa 2", "Geral 2", "Logo 2", "Sexo 2"])


  confrontos['Logo 1'] = confrontos['Logo 1'].apply(lambda x: path_to_image_html(x))
  confrontos['Logo 2'] = confrontos['Logo 2'].apply(lambda x: path_to_image_html(x))

  sprteio_html = confrontos.to_html(escape=False)

  with open("out.html", "w", encoding="utf-8") as file:
    file.writelines('<meta charset="UTF-8">\n')
    file.write(sprteio_html)

  def get_options():
    return {
        'encoding': 'UTF-8',
        'enable-local-file-access': True
    }


  pdfkit.from_file('out.html', 'out.pdf',options=get_options())

  #telegram_bot.send_file('out.html')

  telegram_bot.send_file()

  return confrontos





In [ ]:
teams,distance_array = get_teams(teams_shape=50)


?page=23


In [ ]:
sorteio = sorteia(teams,distance_array,threshold = 10)

-1001833850465
{"ok":true,"result":{"message_id":13,"from":{"id":6184052476,"is_bot":true,"first_name":"FIFA","username":"fifa_sette_sorteio_bot"},"chat":{"id":-1001833850465,"title":"Sorteio do calvo EA 24","type":"supergroup"},"date":1690152150,"document":{"file_name":"out.pdf","mime_type":"application/pdf","thumbnail":{"file_id":"AAMCAQADHQRtTlZhAAMNZL2s1oeExWknsVdAjLv6065lXUIAAoQDAAKcSvFFPLnRT7YrtZABAAdtAAMvBA","file_unique_id":"AQADhAMAApxK8UVy","file_size":13624,"width":226,"height":320},"thumb":{"file_id":"AAMCAQADHQRtTlZhAAMNZL2s1oeExWknsVdAjLv6065lXUIAAoQDAAKcSvFFPLnRT7YrtZABAAdtAAMvBA","file_unique_id":"AQADhAMAApxK8UVy","file_size":13624,"width":226,"height":320},"file_id":"BQACAgEAAx0EbU5WYQADDWS9rNaHhMVpJ7FXQIy7-tOuZV1CAAKEAwACnErxRTy50U-2K7WQLwQ","file_unique_id":"AgADhAMAApxK8UU","file_size":195114},"caption":"Sorteio do fifa 23"}}


In [ ]:
sorteio_html = HTML(sorteio.to_html(escape=False))

In [ ]:
sorteio_html

,Nome 1,Liga 1,Ataque 1,Meio 1,Defesa 1,Geral 1,Logo 1,Sexo 1,Nome 2,Liga 2,Ataque 2,Meio 2,Defesa 2,Geral 2,Logo 2,Sexo 2
0,Leverkusen,Bundesliga,80,79,80,79,,M,Sevilla FC,LaLiga Santander,79,82,79,81,,M
1,Milan,Serie A TIM,80,81,81,81,,M,Napoli FC,Serie A TIM,80,81,80,80,,M
2,Athletic Club,LaLiga Santander,80,78,79,79,,M,Juventus,Serie A TIM,82,81,81,82,,M
3,Chelsea,Premier League,83,79,83,83,,M,Sporting CP,Liga Portugal,77,79,78,78,,M
4,M'gladbach,Bundesliga,79,78,76,77,,M,Aston Villa,Premier League,78,79,79,79,,M
5,Galatasaray,Süper Lig,79,76,74,76,,M,RC Celta,LaLiga Santander,77,77,77,77,,M
6,Galatasaray,Süper Lig,79,76,74,76,,M,Rayo Vallecano,LaLiga Santander,79,75,76,76,,M
7,Roma,Serie A TIM,82,77,81,80,,M,AS Monaco,Ligue 1 Uber Eats,81,77,77,77,,M
8,RB Leipzig,Bundesliga,83,81,80,81,,M,Sevilla FC,LaLiga Santander,79,82,79,81,,M
9,Athletic Club,LaLiga Santander,80,78,79,79,,M,SL Benfica,Liga Portugal,80,79,78,79,,M


In [ ]:
random.randint(0, 10)

7

In [ ]:
open('out.pdf','rb')

<_io.BufferedReader name='out.pdf'>